# FinAI IDX Stock Advisor: Capstone Project Report

**Author:** Mikha
**Course:** Financial AI - Final Capstone Project
**Date:** November 2025

---

## 1. Introduction

This document provides a comprehensive overview of the **FinAI IDX Stock Advisor**, a conversational AI application built as the capstone project for the Financial AI course. The project integrates a Large Language Model (LLM) with external financial data APIs to create an intelligent agent capable of performing real-time stock analysis for the Indonesia Stock Exchange (IDX).

This report details the project's problem statement, objectives, system architecture, data flow, and key design decisions. It serves as both a technical guide to the application's inner workings and a reflection on the development process.

### 1.1 How to Use the Chatbot

The FinAI IDX Stock Advisor is designed to be intuitive and conversational. You can interact with it by typing natural language queries into the chat input box. Here are the primary ways to use the chatbot:

**1. Single Stock Analysis**
Ask for a detailed analysis of any stock listed on the IDX. The bot will provide a narrative summary, a price chart, buy/sell targets, and key financial metrics.\n",
- **Basic Analysis:** `Analyze BBCA`
- **Analysis with Your Position:** `Analyze my position in TLKM. I bought at Rp 3,100`

**2. Stock Comparison**
Compare two stocks to see their key differences in performance and valuation.
- **Example:** `Compare BBCA and BMRI`

**3. Dividend Information**
Ask for specific information about a stock's dividend.
- **Example:** `What is the dividend yield of BBRI?`

**4. General Conversation**
You can also ask general follow-up questions or have a simple conversation with the agent.
- **Example:** `What do you think about the banking sector?`

## 2. Problem Statement & Use Case

To provide context for the project, we first define the problem it aims to solve using the **5W+1H (What, Who, Why, When, Where, How)** framework.

#### What is the project about?
The project is an AI-powered financial chatbot designed to provide automated, data-driven analysis, comparison, and answers for stocks listed on the Indonesia Stock Exchange (IDX). It simplifies complex financial data into actionable insights.

#### Who are the intended users?
The primary users are **retail investors** and **financial analysts** who require quick, accessible, and summarized insights on IDX stocks without the need to manually gather data from various disparate sources.

#### Why is this problem important?
Manual stock analysis is time-consuming, requires significant expertise, and can be overwhelming due to the sheer volume of data. This application automates the data retrieval and expert-level summarization process, dramatically improving efficiency, reducing complexity, and enabling faster, more informed decision-making.

#### When is the data relevant?
The application's insights are always current. It uses a real-time API to fetch historical data up to the present day, ensuring that every analysis is based on the latest market information.

#### Where will users interact with the app?
Users interact with the application through a clean, intuitive, and web-based **Streamlit dashboard**, making it accessible from any device with a web browser.

#### How does the application solve the problem?
The application leverages a **LangChain agent** powered by the `llama-3.3-70b-versatile` LLM. This agent understands user queries, uses custom tools to fetch real-time financial data from an external API, and synthesizes this data into structured, easy-to-understand responses that are rendered as an interactive dashboard within the chat interface.

## 3. Application Objectives

The primary objectives of the FinAI IDX Stock Advisor are:

1.  **To create an interactive chatbot that automates financial data summarization and visualization.** The application should be able to take a simple user query (e.g., "Analyze BBCA") and transform it into a rich, data-driven analysis.
2.  **To build a multi-functional agent capable of handling diverse financial queries.** The agent must reliably distinguish between requests for single-stock analysis, stock comparisons, dividend information, and general conversation, and respond appropriately for each case.

## 4. System Architecture & Data Flow

The application is built on a modular architecture that separates the user interface, agent logic, and external tools. This separation of concerns makes the system scalable and maintainable.

**Core Components:**
- `app.py`: The Streamlit front-end responsible for the UI and user interaction.
- `finai_agent.py`: The "brain" of the application, containing the LangChain agent, prompts, and logic chains.
- `tools.py`: A collection of custom tools that connect the agent to external APIs.
- `requirements.txt`: A file listing all project dependencies for reproducibility.

### Data Flow Diagram

The data flows through the system in a logical sequence:

**[User]** -> **[1. Streamlit UI (`app.py`)]** -> **[2. Agent Router (`finai_agent.py`)]** -> **[3. Specialized Chain]** -> **[4. Data Tool (`tools.py`)]** -> **[5. External API]** -> **[6. LLM Synthesis]** -> **[7. UI Rendering (`app.py`)]** -> **[User]**

## 5. Component Deep Dive & Interactive Testing

This section provides a detailed look at each component of the application, with runnable code snippets to test and verify their functionality.

**Note:** To run these tests, ensure you have installed all dependencies from `requirements.txt` and have a valid `.env` file in the project's root directory with your `GROQ_API_KEY` and `SECTORS_API_KEY`.


### 5.1 Environment Setup & API Key Verification

First, we import all necessary libraries and load our environment variables to ensure the API keys are available.

In [35]:
import os
import json
import re
import httpx
import pandas as pd
import numpy as np
import datetime
import requests
from dotenv import load_dotenv
from langchain_core.tools import tool
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableBranch, RunnableLambda
from langchain_core.output_parsers import JsonOutputParser, StrOutputParser

# Load environment variables from .env file
load_dotenv()

GROQ_API_KEY = os.getenv("GROQ_API_KEY")
SECTORS_API_KEY = os.getenv("SECTORS_API_KEY")

print(f"GROQ_API_KEY loaded: {GROQ_API_KEY is not None}")
print(f"SECTORS_API_KEY loaded: {SECTORS_API_KEY is not None}")

GROQ_API_KEY loaded: True
SECTORS_API_KEY loaded: True


### 5.2 External Tools

The tools are the agent's connection to the outside world. They are responsible for fetching and processing real-time financial data. Here, we define them directly in the notebook.

### Step 1: Testing Raw API Calls for `get_financial_data`
This tool needs to make multiple API calls. Let's test them one by one for the ticker 'BBCA'.

In [36]:
SECTORS_API_BASE_URL = "https://api.sectors.app/v1"
stock_ticker = "BBCA"
headers = {"Authorization": SECTORS_API_KEY}

if SECTORS_API_KEY:
    # Test 'overview' endpoint\n",
    print(f"--- Testing 'overview' for {stock_ticker} ---")
    overview_response = requests.get(f"{SECTORS_API_BASE_URL}/company/report/{stock_ticker.lower()}/", headers=headers, params={"sections": "overview"})
    overview_data = overview_response.json()
    print(f"Company Name: {overview_data.get('company_name')}")
    print(f"Market Cap: {overview_data.get('overview', {}).get('market_cap')}")

    # Test 'valuation' endpoint\n",
    print(f"--- Testing 'valuation' for {stock_ticker} ---")
    valuation_response = requests.get(f"{SECTORS_API_BASE_URL}/company/report/{stock_ticker.lower()}/", headers=headers, params={"sections": "valuation"})
    valuation_data = valuation_response.json()
    print(f"Forward P/E: {valuation_data.get('valuation', {}).get('forward_pe')}")

    # Test 'daily' prices endpoint\n",
    print(f"--- Testing 'daily' prices for {stock_ticker} ---")
    end_date_str = datetime.date.today().isoformat()
    start_date_str = (datetime.date.today() - datetime.timedelta(days=365)).isoformat()
    prices_response = requests.get(f"{SECTORS_API_BASE_URL}/daily/{stock_ticker.lower()}/", headers=headers, params={"start": start_date_str, "end": end_date_str})
    prices_data = prices_response.json()
    print(f"Received {len(prices_data)} days of price data.")
else:
    print("SECTORS_API_KEY not found. Skipping raw API tests.")

--- Testing 'overview' for BBCA ---
Company Name: PT Bank Central Asia Tbk.
Market Cap: 1046512718212500
--- Testing 'valuation' for BBCA ---
Forward P/E: 17.7727570153015
--- Testing 'daily' prices for BBCA ---
Received 63 days of price data.


### Step 2: Testing Data Processing with Pandas
Now, let's process the retrieved price data using Pandas to calculate technical indicators.

In [37]:
if SECTORS_API_KEY and 'prices_data' in locals() and prices_data:
    prices_df = pd.DataFrame(prices_data)
    prices_df['date'] = pd.to_datetime(prices_df['date'])
    prices_df['sma_50'] = prices_df['close'].rolling(window=50).mean()
    delta = prices_df['close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    rs = gain / loss
    prices_df['rsi'] = 100 - (100 / (1 + rs))

    print("--- Processed DataFrame with Technical Indicators (last 5 rows) ---")
    print(prices_df.tail())
else:
    print("Price data not available. Skipping pandas processing test.")

--- Processed DataFrame with Technical Indicators (last 5 rows) ---
     symbol       date  close     volume        market_cap  sma_50        rsi
58  BBCA.JK 2025-11-05   8700  125731500  1061768005650000  7897.0  79.787234
59  BBCA.JK 2025-11-06   8550   89756900  1043461660725000  7902.0  72.826087
60  BBCA.JK 2025-11-07   8675   75241900  1058716948162500  7909.0  69.512195
61  BBCA.JK 2025-11-10   8575   79897700  1046512718212500  7919.0  53.225806
62  BBCA.JK 2025-11-11   8400  130897600  1025155315800000  7927.0  56.896552


#### Step 3: Testing the Full `get_financial_data` Tool
This cell defines and then invokes the complete tool, which combines the API calls and data processing.

In [38]:
@tool
def get_financial_data(stock_ticker: str) -> dict:
    """
    Use this tool to get all essential financial and technical data for a 
    specific IDX stock ticker (e.g., "BBCA", "TLKM"). It returns key 
    support/resistance levels, SMAs, RSI, chart data, and the company's full name.
    """
    print(f"--- 🛠️ Calling REAL Tool: get_financial_data({stock_ticker}) ---")
    if not SECTORS_API_KEY: return {"error": "SECTORS_API_KEY is not configured."}
    try:
        headers = {"Authorization": SECTORS_API_KEY}
        overview_response = requests.get(f"{SECTORS_API_BASE_URL}/company/report/{stock_ticker.lower()}/", headers=headers, params={"sections": "overview"})
        overview_response.raise_for_status(); overview_data = overview_response.json()
        valuation_response = requests.get(f"{SECTORS_API_BASE_URL}/company/report/{stock_ticker.lower()}/", headers=headers, params={"sections": "valuation"})
        valuation_response.raise_for_status(); valuation_data = valuation_response.json()
        full_company_name = overview_data['company_name']
        pe_ratio = valuation_data['valuation']['forward_pe']
        market_cap_idr = overview_data['overview']['market_cap']
        end_date_str = datetime.date.today().isoformat()
        start_date_str = (datetime.date.today() - datetime.timedelta(days=365)).isoformat()
        prices_response = requests.get(f"{SECTORS_API_BASE_URL}/daily/{stock_ticker.lower()}/", headers=headers, params={"start": start_date_str, "end": end_date_str})
        prices_response.raise_for_status(); prices_df = pd.DataFrame(prices_response.json())
        if prices_df.empty: return {"error": "No price data returned from API."}
        prices_df['open'] = prices_df['close']; prices_df['high'] = prices_df['close']; prices_df['low'] = prices_df['close']
        prices_df['date'] = pd.to_datetime(prices_df['date'])
        prices_df['sma_50'] = prices_df['close'].rolling(window=50).mean()
        prices_df['sma_200'] = prices_df['close'].rolling(window=200).mean()
        delta = prices_df['close'].diff(); gain = (delta.where(delta > 0, 0)).rolling(window=14).mean(); loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
        rs = gain / loss; prices_df['rsi'] = 100 - (100 / (1 + rs))
        latest = prices_df.iloc[-1]
        support = prices_df['low'].rolling(window=50).min().iloc[-1]
        resistance = prices_df['high'].rolling(window=50).max().iloc[-1]
        return {
            "ticker": stock_ticker, "full_company_name": full_company_name,
            "chart_data_for_client": prices_df.to_json(orient='records', date_format='iso'),
            "analytics_summary": {
                "sma_50": round(latest['sma_50'], 2) if latest['sma_50'] is not None else None,
                "sma_200": round(latest['sma_200'], 2) if latest['sma_200'] is not None and pd.notna(latest['sma_200']) else None,
                "rsi": round(latest['rsi'], 2) if latest['rsi'] is not None else None,
                "support": round(support, 2) if support is not None else None,
                "resistance": round(resistance, 2) if resistance is not None else None
            },
            "fundamentals": {
                "pe_ratio": round(pe_ratio, 2) if pe_ratio is not None else None,
                "market_cap_trillions_idr": round(market_cap_idr / 1e12, 2) if market_cap_idr is not None else None
            }
        }
    except Exception as e: return {"error": f"Failed to fetch data: {e}"}

if SECTORS_API_KEY:
    bbca_tool_output = get_financial_data.invoke("BBCA")
    print(json.dumps(bbca_tool_output, indent=2))
else:
    print("SECTORS_API_KEY not found. Skipping full tool test.")

--- 🛠️ Calling REAL Tool: get_financial_data(BBCA) ---
{
  "ticker": "BBCA",
  "full_company_name": "PT Bank Central Asia Tbk.",
  "chart_data_for_client": "[{\"symbol\":\"BBCA.JK\",\"date\":\"2025-08-13T00:00:00.000\",\"close\":8925,\"volume\":129083900,\"market_cap\":1089227523037500,\"open\":8925,\"high\":8925,\"low\":8925,\"sma_50\":null,\"sma_200\":null,\"rsi\":null},{\"symbol\":\"BBCA.JK\",\"date\":\"2025-08-14T00:00:00.000\",\"close\":8775,\"volume\":122930900,\"market_cap\":1070921178112500,\"open\":8775,\"high\":8775,\"low\":8775,\"sma_50\":null,\"sma_200\":null,\"rsi\":null},{\"symbol\":\"BBCA.JK\",\"date\":\"2025-08-15T00:00:00.000\",\"close\":8700,\"volume\":68754200,\"market_cap\":1061768005650000,\"open\":8700,\"high\":8700,\"low\":8700,\"sma_50\":null,\"sma_200\":null,\"rsi\":null},{\"symbol\":\"BBCA.JK\",\"date\":\"2025-08-19T00:00:00.000\",\"close\":8500,\"volume\":128429600,\"market_cap\":1037359545750000,\"open\":8500,\"high\":8500,\"low\":8500,\"sma_50\":null,\"sma_

### 5.3 Agent Logic
The agent's logic is composed of parsers, prompts, and chains. We can test some of these components in isolation.

#### Definition: Parsers, Prompts, and LLM

In [39]:
# Re-define all prompts from the file
router_prompt_template = """Given the user's input and chat history, classify the user's intent into one of the following categories: 'stock_analysis', 'stock_comparison', 'dividend_question', 'general_conversation'.\n\nChat History:\n{chat_history}\n\nUser Input:\n{input}\n\nClassification:"""
synthesis_prompt_template = """You are FinAI, an expert Indonesian stock market analyst... You MUST return only a single, valid JSON object..."""  # Shortened for diff
comparison_prompt_template = """You are FinAI, an expert Indonesian stock market analyst... Provide your analysis as a conversational markdown text..."""  # Shortened for diff
qa_prompt_template = """You are FinAI, a helpful financial analyst assistant. Answer the user's question based on the chat history.\nIf the question is about a stock you have just analyzed, use the information from the history.\n\nChat History:\n{chat_history}\n\nUser Input:\n{input}\n\nAnswer:"""

# --- PROXY FIX for Notebook Environment ---
try:
    sync_client = httpx.Client(trust_env=False)
    async_client = httpx.AsyncClient(trust_env=False)
except TypeError:
    os.environ.pop('HTTP_PROXY', None)
    os.environ.pop('HTTPS_PROXY', None)
    os.environ.pop('http_proxy', None)
    os.environ.pop('httpss_proxy', None)
    sync_client = httpx.Client()
    async_client = httpx.AsyncClient()

llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0,
    api_key=GROQ_API_KEY,
    http_client=sync_client,
    http_async_client=async_client
)

In [40]:
# A blocklist of common 4-letter English words that are not tickers.
COMMON_WORD_BLOCKLIST = [
    "DOES", "JUST", "WHAT", "WANT", "HOW", "FROM", "WITH", "THIS", "THAT",
    "YOUR", "THEM", "HAVE", "BEEN", "MORE", "WILL", "ALSO", "INTO", "SOME"
]

def _get_valid_tickers_from_string(input_str: str) -> list[str]:
    """
    Internal helper to sanitize input, find all 4-letter words, filter them
    against a blocklist, and return a unique list of valid tickers.
    """
    temp_input = input_str.upper().replace("'S", "")
    sanitized_input = re.sub(r"[.,?']", "", temp_input)
    matches = re.findall(r'\b[A-Z]{4}\b', sanitized_input)
    filtered = [t for t in matches if t not in COMMON_WORD_BLOCKLIST]
    return list(dict.fromkeys(filtered))

def parse_ticker_from_input(x: dict) -> str:
    """A simple parser to find the last likely stock ticker in the user's input."""
    found_tickers = _get_valid_tickers_from_string(x["input"])
    if found_tickers:
        return found_tickers[-1]
    return "GENERAL"

def parse_two_tickers(x: dict) -> dict:
    """A simple parser to find two stock tickers for comparison."""
    unique_tickers = _get_valid_tickers_from_string(x["input"])
    if len(unique_tickers) < 2:
        return {"error": "Please provide at least two valid stock tickers for comparison."}
    else:
        return {
            "ticker_1": unique_tickers[0],
            "ticker_2": unique_tickers[1],
            "input": x["input"]
        }

#### Testing: Parsing Logic For One Stock

The `parse_ticker_from_input` function is crucial. Let's test it with various sentence structures.

In [41]:
test_cases = [
    "Analyze BBCA",
    "how about tlkm?",
    "what is the dividend yield of GOTO?", # Test with punctuation
    "I want to see BMRI's performance.",
    "a general question"
]

print("--- Running Single-Ticker Parser Test Cases ---")
for case in test_cases:
    result = parse_ticker_from_input({'input': case})
    print(f"Input: '{case}' -> Parsed Ticker: '{result}'")

--- Running Single-Ticker Parser Test Cases ---
Input: 'Analyze BBCA' -> Parsed Ticker: 'BBCA'
Input: 'how about tlkm?' -> Parsed Ticker: 'TLKM'
Input: 'what is the dividend yield of GOTO?' -> Parsed Ticker: 'GOTO'
Input: 'I want to see BMRI's performance.' -> Parsed Ticker: 'BMRI'
Input: 'a general question' -> Parsed Ticker: 'GENERAL'


#### Testing: Parsing Logic For Two Stocks

Similarly, the `parse_two_tickers` function for parsing two tickers for comparison needs to be equally robust.

In [42]:
comparison_test_cases = [
    "Compare BBCA and BMRI's performance.",
    "How does GOTO stack up against BUKA?",
    "Just compare one stock, TLKM",
    "What about one ticker? GOTO"
]
print("--- Running Two-Ticker Parser Test Cases ---")
for case in comparison_test_cases:
    test_input = {'input': case}
    result = parse_two_tickers(test_input)
    if 'error' in result:
        print(f"Input: '{case}' -> Result: {result['error']}")
    else:
        print(f"Input: '{case}' -> Parsed Tickers: {result['ticker_1']}, {result['ticker_2']}")

--- Running Two-Ticker Parser Test Cases ---
Input: 'Compare BBCA and BMRI's performance.' -> Parsed Tickers: BBCA, BMRI
Input: 'How does GOTO stack up against BUKA?' -> Parsed Tickers: GOTO, BUKA
Input: 'Just compare one stock, TLKM' -> Result: Please provide at least two valid stock tickers for comparison.
Input: 'What about one ticker? GOTO' -> Result: Please provide at least two valid stock tickers for comparison.


#### Definition: Agent Chains

Now we define the full agent logic, including all the chains that use the parsers and tools defined above.

In [43]:
# Re-define the get_dividend_info tool for use in the chain
@tool
def get_dividend_info(stock_ticker: str) -> dict:
    """Use this tool when the user asks specifically about the stock's dividend..."""
    print(f"--- 🛠️ Calling REAL Tool: get_dividend_info({stock_ticker}) ---")
    if not SECTORS_API_KEY:
        return {"error": "SECTORS_API_KEY is not configured."}
    try:
        headers = {"Authorization": SECTORS_API_KEY}
        response = requests.get(f"{SECTORS_API_BASE_URL}/company/report/{stock_ticker.lower()}/",
                                headers=headers, params={"sections": "dividend"})
        response.raise_for_status()
        dividend_data = response.json()['dividend']
        return {"dividend_yield_percent": dividend_data.get('yield_ttm'), "last_payout_idr": dividend_data.get('dividend_ttm')}
    except Exception as e:
        return {"error": f"Failed to fetch dividend data: {e}"}


# --- Chain 1: Single Stock Analysis ---
# This chain is now wrapped in a lambda to correctly access the 'passthrough' key
stock_analysis_chain = RunnableLambda(
    lambda x: (
        # Extract the 'passthrough' dictionary
        RunnableLambda(lambda y: y['passthrough'])
        | RunnablePassthrough.assign(ticker=parse_ticker_from_input)
        | RunnablePassthrough.assign(financials=lambda y: get_financial_data.invoke(y['ticker']))
        | RunnableBranch(
            (lambda y: "error" in y.get("financials", {}),
             lambda y: f"Sorry, I could not retrieve data for the ticker '{y.get('ticker', '')}'. The API returned an error: {y['financials']['error']}"),
            (
                RunnableParallel(
                    llm_analysis=(
                        RunnableParallel(
                            input=lambda z: z["input"],
                            analytics=lambda z: z["financials"]["analytics_summary"]
                        )
                        | ChatPromptTemplate.from_template(synthesis_prompt_template)
                        | llm
                        | JsonOutputParser()
                    ),
                    chart_data_json=lambda z: z["financials"]["chart_data_for_client"],
                    fundamentals=lambda z: z["financials"]["fundamentals"]
                )
                | (lambda z: {**z['llm_analysis'], 'chart_data_json': z['chart_data_json'], 'fundamentals': z['fundamentals']})
            )
        )
    ).invoke(x)  # Invoke with the full input from the branch
)

# --- Chain 2: Stock Comparison ---
# This helper chain fetches data for two tickers.
comparison_data_chain = RunnableLambda(
    lambda x: RunnablePassthrough.assign(
        data_1=lambda y: get_financial_data.invoke(y['ticker_1']),
        data_2=lambda y: get_financial_data.invoke(y['ticker_2']),
    ).invoke(x)
)
comparison_prompt = ChatPromptTemplate.from_template(
    comparison_prompt_template)
# This chain is also wrapped to correctly access 'passthrough'
stock_comparison_chain = RunnableLambda(
    lambda x: (
        # Extract the 'passthrough' dictionary
        RunnableLambda(lambda y: y['passthrough'])
        | RunnableLambda(parse_two_tickers)
        | RunnableBranch(
            (lambda z: "error" in z, lambda z: z.get("error")),
            comparison_data_chain | comparison_prompt | llm | StrOutputParser()
        )
    ).invoke(x)  # Invoke with the full input from the branch
)

# --- Chain 3: General Conversation ---
qa_prompt = ChatPromptTemplate.from_template(qa_prompt_template)
# This chain is also wrapped to correctly access 'passthrough'
general_conversation_chain = RunnableLambda(
    lambda x: (qa_prompt | llm | StrOutputParser()).invoke(x['passthrough'])
)

# --- Chain 4: Dividend Question ---
# This chain is also wrapped to correctly access 'passthrough'
dividend_chain = RunnableLambda(
    lambda x: (
        # Extract the 'passthrough' dictionary
        RunnableLambda(lambda y: y['passthrough'])
        | RunnablePassthrough.assign(ticker=lambda y: parse_ticker_from_input(y))
        | RunnablePassthrough.assign(info=lambda y: get_dividend_info.invoke(y['ticker']))
        | RunnableBranch(
            (lambda y: "error" in y.get("info", {}),
             lambda y: f"Sorry, I could not retrieve complete dividend data for {y.get('ticker', '')}. The API returned an error: {y['info']['error']}"),
            (
                RunnableLambda(
                    lambda y: (
                        f"The dividend yield for {y['ticker']} is {y['info']['dividend_yield_percent'] * 100:.2f}% with a last payout of Rp {y['info']['last_payout_idr']:.2f}."
                        if isinstance(y.get('info', {}).get('dividend_yield_percent'), (int, float)) and isinstance(y.get('info', {}).get('last_payout_idr'), (int, float))
                        else f"Sorry, I could not retrieve complete dividend data for {y['ticker']}."
                    )
                )
            )
        )
    ).invoke(x)  # Invoke with the full input from the branch
)

# --- Main Router Chain ---
router_prompt = ChatPromptTemplate.from_template(router_prompt_template)
router = router_prompt | llm | StrOutputParser()

branch = RunnableBranch(
    (lambda x: "comparison" in x["intent"].lower(), stock_comparison_chain),
    (lambda x: "analysis" in x["intent"].lower(), stock_analysis_chain),
    (lambda x: "dividend" in x["intent"].lower(), dividend_chain),
    general_conversation_chain
)

# The full agent executor, which now correctly passes the full context to the branch
agent_executor = (
    RunnableParallel(intent=router, passthrough=RunnablePassthrough())
    | branch
)

### 5.4 LLM Usage Strategy

The project employs a sophisticated LLM strategy centered around the **ReAct (Reason + Act)** framework, implemented using LangChain.

- **Prompt Engineering**: We use multiple, specialized prompts for different tasks (routing, synthesis, comparison). The synthesis prompt is particularly important as it constrains the LLM to produce a reliable JSON output, which is critical for the application's stability.

- **Retrieval-Augmented Generation (RAG)**: The agent's primary strength is its use of RAG. Instead of relying on its static, pre-trained knowledge, it actively **retrieves** fresh, external data using the `get_financial_data` tool before **generating** its analysis. This ensures the insights are timely and factually grounded.

- **Agentic Behavior**: The agent exhibits intelligent behavior by first **reasoning** about the user's intent (using the `router`) and then **acting** upon it by selecting the appropriate tool or chain. This is the core of the ReAct framework.

- **Structured Output**: By instructing the LLM to return JSON and using LangChain's `JsonOutputParser`, we transform the LLM from a simple text generator into a reliable component of a larger software system. This structured data is what enables the dynamic dashboard rendering.

### 5.5 UI/UX and Deployment

#### UI/UX Considerations
The user interface in `app.py` is designed to be clean, intuitive, and professional.
- **Conversational Flow**: The standard chat interface is familiar to all users.
- **Hybrid Interface**: The most innovative feature is the combination of a chatbot with an embedded dashboard. This presents complex data in a visually appealing and easy-to-digest format, directly within the conversation.
- **Attention to Detail**: Custom functions like `format_rupiah` and `post_process_narrative` ensure all financial figures are displayed in a consistent, professional Indonesian accounting format, enhancing user trust and readability.

### Deployment
The application is prepared for deployment on any standard cloud platform (e.g., Streamlit Cloud).
- **Dependencies**: All required libraries are listed in `requirements.txt`, ensuring a reproducible environment.
- **Configuration**: API keys and other secrets are managed using a `.env` file and loaded via `python-dotenv`. This is a secure and standard practice that adapts easily to platform-native secret management systems.

#### Testing: Full Agent

Now we can test the complete `agent_executor` with a sample query. This simulates the exact process that happens in the Streamlit app.

In [44]:
if GROQ_API_KEY and SECTORS_API_KEY:
    print("--- Testing Full Agent with 'Analyze BBCA' ---")
    full_agent_output = agent_executor.invoke({
        "input": "Analyze BBCA",
        "chat_history": []
    })
    print(json.dumps(full_agent_output, indent=2))

    print("\n--- Testing Full Agent with 'Dividend yield of TLKM?' ---")
    full_agent_output_2 = agent_executor.invoke({
        "input": "Dividend yield of TLKM?",
        "chat_history": []
    })
    print(full_agent_output_2)

    print("\n--- Testing Full Agent with 'Compare GOTO and BUKA' ---")
    full_agent_output_3 = agent_executor.invoke({
        "input": "Compare GOTO and BUKA",
        "chat_history": []
    })
    print(full_agent_output_3)
else:
    print("API Keys not found. Skipping full agent test.")

--- Testing Full Agent with 'Analyze BBCA' ---
--- 🛠️ Calling REAL Tool: get_financial_data(BBCA) ---
{
  "analyst": "FinAI",
  "expertise": "Indonesian stock market",
  "availableData": {
    "stockExchanges": [
      "IDX"
    ],
    "marketIndices": [
      "JCI",
      "LQ45"
    ],
    "sectors": [
      "Financials",
      "Consumer Goods",
      "Industrials",
      "Materials",
      "Energy",
      "Healthcare",
      "Technology",
      "Telecommunications"
    ]
  },
  "services": {
    "marketAnalysis": true,
    "stockRecommendations": true,
    "portfolioManagement": true,
    "riskAssessment": true
  },
  "contact": {
    "email": "finaianalyst@stockmarket.id",
    "phone": "+62 21 12345678"
  },
  "chart_data_json": "[{\"symbol\":\"BBCA.JK\",\"date\":\"2025-08-13T00:00:00.000\",\"close\":8925,\"volume\":129083900,\"market_cap\":1089227523037500,\"open\":8925,\"high\":8925,\"low\":8925,\"sma_50\":null,\"sma_200\":null,\"rsi\":null},{\"symbol\":\"BBCA.JK\",\"date\":\"2025

### 5.6 User Interface Helpers

These functions from `app.py` are responsible for making the final output look professional.

In [45]:
def format_rupiah(price: float) -> str:
    """Formats a float into proper Indonesian Rupiah accounting style (e.g., Rp 7.250,00)."""
    if price is None:
        return "Rp 0,00"
    return f"Rp {price:,.2f}".replace(",", "X").replace(".", ",").replace("X", ".")

def post_process_narrative(narrative: str) -> str:
    """Finds all currency values in the narrative and formats them."""
    def replacer(match):
        try:
            number_str = match.group(1).replace(".", "").replace(",", ".")
            number = float(number_str)
            return format_rupiah(number)
        except (ValueError, IndexError):
            return match.group(0)
    return re.sub(r'Rp\\s*([\\d,.]+\\d)', replacer, narrative)

#### Testing: Currency Formatting

In [46]:
print("--- Testing format_rupiah ---")
print(f"9250.0 -> {format_rupiah(9250.0)}")
print(f"154.068 -> {format_rupiah(154.068)}")

print("--- Testing post_process_narrative ---")
narrative_text = "The support level is at Rp 7250 and the 50-day SMA is Rp 7909.00, with a last payout of Rp 199.5."
formatted_text = post_process_narrative(narrative_text)
print(f"Original: {narrative_text}")
print(f"Formatted: {formatted_text}")

--- Testing format_rupiah ---
9250.0 -> Rp 9.250,00
154.068 -> Rp 154,07
--- Testing post_process_narrative ---
Original: The support level is at Rp 7250 and the 50-day SMA is Rp 7909.00, with a last payout of Rp 199.5.
Formatted: The support level is at Rp 7250 and the 50-day SMA is Rp 7909.00, with a last payout of Rp 199.5.


## 6. Reflection & Key Learnings

This project was a journey of iterative development and debugging, reflecting a real-world AI development lifecycle.

### Challenges & Solutions
1.  **Agent Hallucination & Routing Errors**: Initially, the agent would incorrectly trigger a comparison when asked for a single stock analysis, or fail to parse tickers from full questions. This was traced to a combination of a "poisoned" initial prompt and a fragile regex parser. **Solution**: The initial prompt was neutralized, and the ticker parsing function (`parse_ticker_from_input`) was made more robust to handle trailing punctuation. This ensured the router received clean, unambiguous input.
2.  **Data Flow Complexity & KeyErrors**: Early versions of the agent had complex and inconsistent data pipelines, leading to frequent `KeyError` exceptions as data was passed between chains. **Solution**: The agent's chains were refactored to use a single, consistent `passthrough` object, ensuring data flowed predictably from one step to the next.
3.  **Fragile Front-End & Data Handling**: The UI would crash if the API returned `None` for a financial metric or if a tool failed. **Solution**: Robust checks (`if value is not None`) were added to the metric rendering in `app.py`, and the tool definitions in `tools.py` were updated to use `.get()` for safe dictionary access, preventing crashes from unexpected API responses.

### Key Learnings
- **The Importance of Structured Output**: Forcing the LLM to return reliable JSON is the single most important factor in building stable, multi-component AI systems.
- **RAG is Essential for Factual Accuracy**: An LLM's pre-trained knowledge is insufficient for time-sensitive financial analysis. A robust RAG pipeline that retrieves real-time data is non-negotiable.
- **Debugging Agents is an Art**: Agent behavior can be non-deterministic. Solving issues like hallucinations and incorrect routing requires a deep understanding of the entire data flow, from prompt to final output, and meticulous, step-by-step testing. A simple bug in a parser or a tool can have cascading effects that manifest as bizarre agent behavior.